In [1]:
import pandas as pd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Cargar el archivo CSV principal
data_file = 'API_4_DS2_es_csv_v2_49994.csv'
data = pd.read_csv(data_file, skiprows=4)

# Convertir el dataframe a formato largo
data_long = pd.melt(data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
                    var_name='Year', value_name='Value')

# Convertir la columna Year a numérica
data_long['Year'] = pd.to_numeric(data_long['Year'], errors='coerce')

# Obtener los nombres únicos de países e indicadores
unique_countries = data_long['Country Name'].dropna().unique().tolist()
unique_indicators = data_long['Indicator Name'].dropna().unique().tolist()

# Crear la aplicación Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1(id='main-title', children="Datos Educativos"),
    html.Div(id='subtitle-container'),  # Contenedor para el subtítulo
    html.Div([
        html.Div([
            html.Label("Selecciona un indicador"),
            dcc.Dropdown(
                id='indicator-dropdown',
                options=[{'label': indicator, 'value': indicator} for indicator in unique_indicators],
                value=unique_indicators[0]
            )
        ], style={'width': '32%', 'display': 'inline-block'}),
        html.Div([
            html.Label("Selecciona países"),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in unique_countries],
                value=unique_countries[:5],
                multi=True
            )
        ], style={'width': '32%', 'display': 'inline-block', 'float': 'right'}),
        html.Div([
            html.Label("Selecciona vista"),
            dcc.Dropdown(
                id='view-dropdown',
                options=[
                    {'label': 'Gráfico de Líneas', 'value': 'line'},
                    {'label': 'Mapa', 'value': 'choropleth'}
                ],
                value='line'
            )
        ], style={'width': '32%', 'display': 'inline-block', 'float': 'right'})
    ]),
    dcc.Graph(id='line-chart'),
    html.Label("Selecciona años", style={'margin-top': '20px'}),
    dcc.RangeSlider(
        id='year-slider',
        min=data_long['Year'].min(),
        max=data_long['Year'].max(),
        step=1,
        value=[data_long['Year'].min(), data_long['Year'].max()],
        marks={str(year): str(year) for year in range(int(data_long['Year'].min()), int(data_long['Year'].max()) + 1, 5)}
    )
])

@app.callback(
    [Output('line-chart', 'figure'),
     Output('subtitle-container', 'children')],
    [Input('indicator-dropdown', 'value'),
     Input('country-dropdown', 'value'),
     Input('year-slider', 'value'),
     Input('view-dropdown', 'value')]
)
def update_chart(selected_indicator, selected_countries, selected_years, selected_view):
    filtered_data = data_long[
        (data_long['Indicator Name'] == selected_indicator) & 
        (data_long['Country Name'].isin(selected_countries)) &
        (data_long['Year'] >= selected_years[0]) & 
        (data_long['Year'] <= selected_years[1])
    ]
    
    if selected_view == 'line':
        fig = px.line(filtered_data, x='Year', y='Value', color='Country Name', 
                      title=f'Indicador: {selected_indicator}')
    elif selected_view == 'choropleth':
        latest_year = filtered_data['Year'].max()
        filtered_data_latest = filtered_data[filtered_data['Year'] == latest_year]
        fig = px.choropleth(filtered_data_latest, locations='Country Name', locationmode='country names', 
                            color='Value', hover_name='Country Name', 
                            title=f'Indicador: {selected_indicator} en {latest_year}')
    
    new_title = f"Visualización de {selected_indicator}"
    subtitle = html.H2(new_title)  
    
    return fig, subtitle

if __name__ == '__main__':
    app.run_server(debug=True)
